In [22]:
# Source: https://gist.github.com/cristianpb/c41d2ac99b8a54818e8b821b1febeca7
# https://cristianpb.github.io/blog/velib-folium

import os
import folium
import pandas as pd
import numpy as np
from folium import plugins
import branca.colormap as cm
from tqdm import tqdm
import matplotlib.pyplot as plt
import json



import conf as cf
from datasets import getStationsInfo


In [100]:
with open(cf.DIRPATH + 'model/velib_model_list.json') as f:
    MODEL_INFO = json.load(f)
STATION_INFO = getStationsInfo()

In [101]:
station_list = [model["station_id"] for model in MODEL_INFO]

In [102]:
df_info = STATION_INFO[STATION_INFO["station_id"].isin(station_list)]
df_info

,station_id,stationCode,name,lon,lat,capacity
5,251039991,14111,Cassini - Denfert-Rochereau,2.336035,48.837526,25
7,2515829865,32017,Basilique,2.358867,48.936269,22
10,100769544,5001,Harpe - Saint-Germain,2.343670,48.851519,45
11,37874517,6003,Saint-Sulpice,2.330808,48.851654,21
15,209063434,33006,André Karman - République,2.385136,48.910399,31
...,...,...,...,...,...,...
1448,1062807847,13123,BNF - Bibliothèque Nationale de France,2.376016,48.835027,42
1450,82402482,10026,Gare de l'Est - Chateau Landon,2.362424,48.879305,59
1453,43247738,18026,Ruisseau - Ordener,2.340145,48.892995,35
1455,368766689,42004,Westermeyer - Paul Vaillant-Couturier,2.396664,48.819116,25


In [114]:
d = { "station_id": [], "y_pred": [] }
for model in MODEL_INFO:
    d["station_id"].append(model["station_id"])
    if model["data"]["y_pred"][0] > 100:
        d["y_pred"].append(100.0)
    elif model["data"]["y_pred"][0] < 0:
        d["y_pred"].append(0.0)
    else:
        d["y_pred"].append(model["data"]["y_pred"][0])
df_data = pd.DataFrame(data = d)
df_data

,station_id,y_pred
0,43247738,9.572862
1,66491386,35.054563
2,100855693,31.021948
3,129050965,29.153207
4,653159330,42.009331
...,...,...
695,15462861,27.583032
696,452341449,9.579583
697,230873471,14.358139
698,39149651,71.604003


In [115]:
velibs = df_info.merge(df_data, on = "station_id")

In [116]:
velibs

,station_id,stationCode,name,lon,lat,capacity,y_pred
0,251039991,14111,Cassini - Denfert-Rochereau,2.336035,48.837526,25,39.095894
1,2515829865,32017,Basilique,2.358867,48.936269,22,51.845901
2,100769544,5001,Harpe - Saint-Germain,2.343670,48.851519,45,45.363019
3,37874517,6003,Saint-Sulpice,2.330808,48.851654,21,21.513080
4,209063434,33006,André Karman - République,2.385136,48.910399,31,14.453020
...,...,...,...,...,...,...,...
695,1062807847,13123,BNF - Bibliothèque Nationale de France,2.376016,48.835027,42,98.102545
696,82402482,10026,Gare de l'Est - Chateau Landon,2.362424,48.879305,59,12.960402
697,43247738,18026,Ruisseau - Ordener,2.340145,48.892995,35,9.572862
698,368766689,42004,Westermeyer - Paul Vaillant-Couturier,2.396664,48.819116,25,100.000000


In [119]:
from colour import Color

red = Color("red")
colors = list(red.range_to(Color("green").hex,10))

def red(brightness):
    #print(brightness)
    brightness = int(round(9 * brightness)) # convert from 0.0-1.0 to 0-255
    return colors[brightness]

m = folium.Map(location=[48.856614, 2.3522219], zoom_start=12, tiles="Stamen Toner")

for k,v in velibs.iterrows():
    folium.CircleMarker(
        location = [v.lat, v.lon],
        fill_color = red(abs(v.y_pred/100)).hex,
        popup = str(v.y_pred) + " % ",
        radius=7
    ).add_to(m)
m

In [76]:
!pip install colour